# Model QCs and figures

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
from scipy import stats
import math
import copy
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rcParams
from matplotlib.ticker import (FormatStrFormatter,MultipleLocator, AutoMinorLocator)


import seppy
import os

sep = seppy.sep()

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# read models

data_axes, data = sep.read_file(datapath+"ch6_data.HH")
pattr = data.reshape(data_axes.n,order='F').T

data_axes, data = sep.read_file(datapath+"ch6un_data_group1.HH")
pattr1 = data.reshape(data_axes.n,order='F').T

data_axes, data = sep.read_file(datapath+"ch6un_data_group2.HH")
pattr2 = data.reshape(data_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6un_model_3d.H")
model0 = data.reshape(axes.n,order='F').T
model0 = model0[:2,:,:,40:81]

axes, data = sep.read_file(datapath+"ch6un_mask.H")
mask = data.reshape(axes.n,order='F').T
mask = mask[0:2,:,:,40:81]

ntr=data_axes.n[1]
ns=data_axes.n[2]
Zn=model0.shape[-1]
Xn=axes.n[1]
Yn=axes.n[2]
Zo=1.92
Xo=axes.o[1]
Yo=axes.o[2]
Zd=axes.d[0]
Xd=axes.d[1]
Yd=axes.d[2]
Xmax=Xo+(Xn-1)*Xd
Ymax=Yo+(Yn-1)*Yd
Zmax=Zo+(Zn-1)*Zd

In [ ]:
# plotting functions

def plotmodel(model, xslice=1.4, yslice=0.05, zslice=1.944, slice_lines=True, vmin=1, vmax=-1, cmap='jet', digits=0, grid=True, pattr=np.array([]), savefig=None):
    "Plot slices of a 3D model"
    
    if vmax<vmin:
        vmin=np.min(model)
        vmax=np.max(model)
    
    ix=int((xslice-Xo)/Xd)
    iy=int((yslice-Yo)/Yd)
    iz=int((zslice-Zo)/Zd)
    cformat = "%."+str(digits)+"f"


#     fig=plt.figure(figsize=(6.66, 6),dpi=300)
    fig=plt.figure(figsize=(12, 6),dpi=300)

    plt.subplot(2,2,1)
    plt.imshow(model[:,:,iz],interpolation='bilinear',extent=[Xo,Xmax,Ymax,Yo],aspect="auto",cmap=cmap,vmin=vmin,vmax=vmax)
    if slice_lines==True:
        plt.hlines(yslice,xmin=Xo,xmax=Xmax,color='c',linestyles='--',linewidths=1)
        plt.vlines(xslice,ymin=Yo,ymax=Ymax,color='c',linestyles='--',linewidths=1)
    
    if pattr.size>0:
        plt.scatter(0.001*pattr[:,0,12],0.001*pattr[:,0,13], s=25, c='m', marker='x')
        plt.scatter(0.001*pattr[:,:,15],0.001*pattr[:,:,16], s=1, c='y', marker=',')
#     plt.xlabel('X (km)')
    plt.gca().set_xticklabels([])
    plt.ylabel('Y (km)')
    plt.gca().invert_yaxis()
    if grid==True:
        plt.grid(color='k', linestyle='-', linewidth=0.2)
    plt.gca().tick_params(axis='both', which='major', labelsize=6)
    plt.title(r'a) Depth = %.2f km' %zslice,loc='left')

    plt.subplot(2,2,3)
    plt.imshow(np.transpose(model[iy,:,:]),interpolation='bilinear',extent=[Xo,Xmax,Zmax,Zo],aspect="auto",cmap=cmap,vmin=vmin,vmax=vmax)
    if slice_lines==True:
        plt.hlines(zslice,xmin=Xo,xmax=Xmax,color='c',linestyles='--',linewidths=1)
        plt.vlines(xslice,ymin=Zo,ymax=Zmax,color='c',linestyles='--',linewidths=1)
    
    if pattr.size>0:
        plt.scatter(0.001*pattr[:,0,12],0.001*pattr[:,0,14], s=25, c='m', marker='x')
        plt.scatter(0.001*pattr[:,:,15],0.001*pattr[:,:,17], s=1, c='y', marker=',')
    plt.xlabel('X (km)')
    plt.ylabel('Depth (km)')
    if grid==True:
        plt.grid(color='k', linestyle='-', linewidth=0.2)
    plt.gca().tick_params(axis='both', which='major', labelsize=6)
    plt.title(r'b) Y = $%.2f$ km' %yslice,loc='left')


    plt.subplot(2,2,4)
    plt.imshow(np.transpose(model[:,ix,:]),interpolation='bilinear',extent=[Yo,Ymax,Zmax,Zo],aspect="auto",cmap=cmap,vmin=vmin,vmax=vmax)
    plt.colorbar(format=cformat,pad=0.01,shrink=0.5,location='right', label="km/s")
    if slice_lines==True:
        plt.hlines(zslice,xmin=Yo,xmax=Ymax,color='c',linestyles='--',linewidths=1)
        plt.vlines(yslice,ymin=Zo,ymax=Zmax,color='c',linestyles='--',linewidths=1)
    
    if pattr.size>0:
        plt.scatter(0.001*pattr[:,0,13],0.001*pattr[:,0,14], s=25, c='m', marker='x')
        plt.scatter(0.001*pattr[:,:,16],0.001*pattr[:,:,17], s=1, c='y', marker=',')
    plt.xlabel('Y (km)')
#     plt.ylabel('Depth (km)')
    plt.gca().set_yticklabels([])
    if grid==True:
        plt.grid(color='k', linestyle='-', linewidth=0.2)
    plt.gca().tick_params(axis='both', which='major', labelsize=6)
    plt.title(r'c) X = %.2f km' %xslice,loc='left')

    plt.tight_layout()
#     plt.tight_layout(pad=0., w_pad=-0.05, h_pad=0.1)

    if savefig != None:
        plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
    else:
        plt.show()
        
        
def plotpert(model, vpmin=1, vpmax=-1, vsmin=1, vsmax=-1, cmap='seismic', digits=2, grid=True, pattr=np.array([]), savefig=None):
    "Plot zslice of Vp and Vs models"
    
    cformat = "%."+str(digits)+"f"
    
    fig=plt.figure(figsize=(6.66, 3),dpi=300)

    plt.subplot(1,2,1)

    if vpmax<vpmin:
        vmax=np.amax(model[0])
        vmin=-vmax
    else:
        vmax=vpmax
        vmin=vpmin
    plt.imshow(model[0,:,:],interpolation='bilinear',extent=[Xo,Xmax,Ymax,Yo],aspect="auto",cmap=cmap,vmin=vmin,vmax=vmax)
    plt.colorbar(format=cformat,pad=0.01,shrink=0.5,label="km/s")
    if pattr.size>0:
        plt.scatter(0.001*pattr[:,0,12],0.001*pattr[:,0,13], s=25, c='m', marker='x')
        plt.scatter(0.001*pattr[:,:,15],0.001*pattr[:,:,16], s=1, c='y', marker=',')
    plt.xlabel('X (km)')
    plt.ylabel('Y (km)')
    plt.gca().invert_yaxis()
    plt.grid(color='k', linestyle='-', linewidth=0.2)
    plt.gca().tick_params(axis='both', which='major', labelsize=6)
    plt.title(r'a) $V_{\mathrm{P}0}$',loc='left')

    plt.subplot(1,2,2)

    if vsmax<vsmin:
        vmax=np.amax(model[1])
        vmin=-vmax
    else:
        vmax=vsmax
        vmin=vsmin
    plt.imshow(model[1,:,:],interpolation='bilinear',extent=[Xo,Xmax,Ymax,Yo],aspect="auto",cmap=cmap,vmin=vmin,vmax=vmax)
    plt.colorbar(format=cformat,pad=0.01,shrink=0.5,label="km/s")
    if pattr.size>0:
        plt.scatter(0.001*pattr[:,0,12],0.001*pattr[:,0,13], s=25, c='m', marker='x')
        plt.scatter(0.001*pattr[:,:,15],0.001*pattr[:,:,16], s=1, c='y', marker=',')
    plt.xlabel('X (km)')
    # plt.ylabel('Y (km)')
    plt.gca().set_yticklabels([])
    plt.gca().invert_yaxis()
    plt.grid(color='k', linestyle='-', linewidth=0.2)
    plt.gca().tick_params(axis='both', which='major', labelsize=6)
    plt.title(r'b) $V_{\mathrm{S}0}$',loc='left')

    plt.tight_layout()
    
    if savefig != None:
        plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
    else:
        plt.show()
        
def plotseis_interleave(d1, d2, inter=50, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=-600, xmax=0, grid=True, colorbar=False, savefig=None):
    """Plot 2D seismic image interleaving traces from 2 datasets"""
 
    assert(d1.shape == d2.shape)
    
    vmin=np.min(d1) / p
    vmax=p*np.max(d1)
    
    if p<0:
        vmax=-p*np.amax(d1)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax

    if d1.ndim==2:
        ns = 1
        nx = d1.shape[0]
        nt = d1.shape[1]
        d1 = np.reshape(d1, (ns,nx,nt))
        d2 = np.reshape(d2, (ns,nx,nt))
    else:
        ns=d1.shape[0]
        nx=d1.shape[1]
        nt=d1.shape[2]

    d = np.copy(d1)
    flag=np.zeros((nx))
    itr1=0
    itr2=inter
    while (itr2<=nx):
        d[:,itr1:itr2,:] = d2[:,itr1:itr2,:]
        flag[itr1:itr2]=0.01
        itr1 += 2*inter
        itr2 += 2*inter
    if itr1<nx:
        d[:,itr1:,:] = d2[:,itr1:,:]
                
    fig=plt.figure(figsize=(8, 6),dpi=300)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=0.)
    for s in range(ns):
        plt.subplot(1,ns,s+1)
        plt.imshow(np.transpose(d[s]),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)   
        plt.plot(np.arange(xmin,xmax+1),flag,linewidth=1,color='r')
        plt.xlabel('Offset (m)')
        plt.gca().tick_params(axis='both', which='major', labelsize=6)
        plt.gca().xaxis.set_minor_locator(MultipleLocator(inter))
        if s==0:
            plt.ylabel('Time (s)')
            plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        else:
            plt.gca().set_yticklabels([])
        if grid==True:
            plt.grid(color='w', linestyle='-', linewidth=0.2, which='both')
    if savefig != None:
        plt.savefig(figpath+str(savefig)+'.png',bbox_inches='tight',format='png')
    else:
        plt.show()

In [ ]:
# plot 3D model
plotmodel(model0[0], pattr=pattr, savefig="ch6un_model_3d")
plotmodel(model0[0], pattr=pattr1, savefig="ch6un_model_3d_group1")
plotmodel(model0[0], pattr=pattr2, savefig="ch6un_model_3d_group2")

In [ ]:
# load and plot 3D models for QC
# axes, data = sep.read_file(datapath+"ch6un_fwi3d_10b.H")
axes, data = sep.read_file(datapath+"ch6un_fwi3d_12b.H")
# axes, data = sep.read_file(datapath+"ch6st_fwi3d_10b.H")
# axes, data = sep.read_file(datapath+"ch6st_fwi3d_12b.H")
model1 = data.reshape(axes.n,order='F').T
model1 = model1[:2,:,:,40:81]

In [ ]:
# plotmodel(model1[0]-model0[0], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.1, vmax=0.1, cmap='seismic', digits=2, pattr=pattr1, savefig="ch6un_fwi3d_10b_vp")
# plotmodel(model1[1]-model0[1], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.25, vmax=0.25, cmap='seismic', digits=2, pattr=pattr1, savefig="ch6un_fwi3d_10b_vs")
plotmodel(model1[0]-model0[0], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.1, vmax=0.1, cmap='seismic', digits=2, pattr=pattr2, savefig="ch6un_fwi3d_12b_vp")
plotmodel(model1[1]-model0[1], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.25, vmax=0.25, cmap='seismic', digits=2, pattr=pattr2, savefig="ch6un_fwi3d_12b_vs")
# plotmodel(model1[0]-model0[0], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.1, vmax=0.1, cmap='seismic', digits=2, pattr=pattr1, savefig="ch6st_fwi3d_10b_vp")
# plotmodel(model1[1]-model0[1], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.25, vmax=0.25, cmap='seismic', digits=2, pattr=pattr1, savefig="ch6st_fwi3d_10b_vs")
# plotmodel(model1[0]-model0[0], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.1, vmax=0.1, cmap='seismic', digits=2, pattr=pattr2, savefig="ch6st_fwi3d_12b_vp")
# plotmodel(model1[1]-model0[1], xslice=1.4,yslice=-0.05, zslice=1.944, slice_lines=True, vmin=-0.25, vmax=0.25, cmap='seismic', digits=2, pattr=pattr2, savefig="ch6st_fwi3d_12b_vs")

In [ ]:
# load and plot stacked perturbations
# axes, data = sep.read_file(datapath+"ch6un_fwi3d_10b.H")
# axes, data = sep.read_file(datapath+"ch6un_fwi3d_12b.H")
# axes, data = sep.read_file(datapath+"ch6st_fwi3d_10b.H")
axes, data = sep.read_file(datapath+"ch6st_fwi3d_12b.H")
model1 = data.reshape(axes.n,order='F').T
model1 = model1[:2,:,:,40:81]

In [ ]:
pert=model1-model0
pert=pert*mask
pert=np.sum(pert,axis=-1)/np.sum(mask,axis=-1)

# plotpert(pert, vpmin=-0.1, vpmax=0.1, vsmin=-0.2, vsmax=0.2, pattr=pattr1, savefig="ch6un_fwi3d_10b_pert")
# plotpert(pert, vpmin=-0.1, vpmax=0.1, vsmin=-0.2, vsmax=0.2, pattr=pattr2, savefig="ch6un_fwi3d_12b_pert")
# plotpert(pert, vpmin=-0.1, vpmax=0.1, vsmin=-0.2, vsmax=0.2, pattr=pattr1, savefig="ch6st_fwi3d_10b_pert")
plotpert(pert, vpmin=-0.1, vpmax=0.1, vsmin=-0.2, vsmax=0.2, pattr=pattr2, savefig="ch6st_fwi3d_12b_pert")

In [ ]:
# similar plots with synthetic models
# axes, data = sep.read_file(datapath+"ch6an_pseudo_model.H")
axes, data = sep.read_file(datapath+"ch6an_fwi3d_1b.H")
model1 = data.reshape(axes.n,order='F').T
model1 = model1[:2,:,:,40:81]

In [ ]:
# plotmodel(model1[0], xslice=1.0,yslice=-0.025, zslice=1.944, slice_lines=True, vmin=1, vmax=-1, digits=0, pattr=pattr1, savefig="ch6an_pseudo_model_vp_group1")
# plotmodel(model1[0]-model0[0], xslice=1.0,yslice=-0.025, zslice=1.944, slice_lines=True, vmin=-0.05, vmax=0.05, cmap='seismic', digits=3, pattr=pattr1, savefig="ch6an_pseudo_model_vp")
# plotmodel(model1[1]-model0[1], xslice=1.0,yslice=-0.025, zslice=1.944, slice_lines=True, vmin=-0.05, vmax=0.05, cmap='seismic', digits=3, pattr=pattr1, savefig="ch6an_pseudo_model_vs")
plotmodel(model1[0]-model0[0], xslice=1.0,yslice=-0.025, zslice=1.944, slice_lines=True, vmin=-0.01, vmax=0.01, cmap='seismic', digits=3, pattr=pattr1, savefig="ch6an_fwi3d_1b_vp")
plotmodel(model1[1]-model0[1], xslice=1.0,yslice=-0.025, zslice=1.944, slice_lines=True, vmin=-0.02, vmax=0.02, cmap='seismic', digits=3, pattr=pattr1, savefig="ch6an_fwi3d_1b_vs")

In [ ]:
pert=model1-model0
pert=pert*mask
pert=np.sum(pert,axis=-1)/np.sum(mask,axis=-1)

# plotpert(pert, vpmin=-0.025, vpmax=0.025, vsmin=-0.025, vsmax=0.025, digits=3, pattr=pattr1, savefig="ch6an_pseudo_model_pert")
plotpert(pert, vpmin=-0.0125, vpmax=0.0125, vsmin=-0.0125, vsmax=0.0125, digits=3, pattr=pattr1, savefig="ch6an_fwi3d_1b_pert")

In [ ]:
# histograms comparison pre- and post-stimulation
axes, data = sep.read_file(datapath+"ch6un_fwi3d_10b.H")
model1 = data.reshape(axes.n,order='F').T
model1 = model1[:2,:,:,40:81]

pert=model1-model0
pert=pert*mask
pert_un=np.sum(pert,axis=-1)/np.sum(mask,axis=-1)

axes, data = sep.read_file(datapath+"ch6st_fwi3d_10b.H")
model1 = data.reshape(axes.n,order='F').T
model1 = model1[:2,:,:,40:81]

pert=model1-model0
pert=pert*mask
pert_st=np.sum(pert,axis=-1)/np.sum(mask,axis=-1)

In [ ]:
istart=350 # x = 1.2 km
iend=650 # x = 1.8 km
bins=np.linspace(-70,80,16)-5

fig=plt.figure(figsize=(8, 6),dpi=300)

plt.subplot(2,1,1)

vals_un=1000*pert_un[0,:,istart:iend].flatten()
vals_st=1000*pert_st[0,:,istart:iend].flatten()
vals_un = vals_un[np.abs(vals_un)>5]
vals_st = vals_st[np.abs(vals_st)>5]

plt.hist(vals_un, bins=bins, density=True, alpha=1, label='Pre-stimulation', color='b')
plt.hist(vals_st, bins=bins, density=True, alpha=0.5, label='Post-stimulation', color='y')
plt.legend()
plt.gca().set_xticklabels([])
plt.ylabel(r'Normalized count')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $V_{\mathrm{P}0}$ perturbation', loc='left')

distance = stats.wasserstein_distance(vals_st,vals_un)
median_distance = np.median(vals_st) - np.median(vals_un)
print("Wasserstein distance between the two distributions is %.3f" %distance)
print("Median distance between the two distributions is %.0f m/s" %median_distance)

plt.subplot(2,1,2)

vals_un=1000*pert_un[1,:,istart:iend].flatten()
vals_st=1000*pert_st[1,:,istart:iend].flatten()
vals_un = vals_un[np.abs(vals_un)>5]
vals_st = vals_st[np.abs(vals_st)>5]

plt.hist(vals_un, bins=bins, density=True, alpha=1, label='Pre-stimulation', color='b')
plt.hist(vals_st, bins=bins, density=True, alpha=0.5, label='Post-stimulation', color='y')
plt.legend()
plt.xlabel(r'Perturbation (m/s)')
plt.ylabel(r'Normalized count')
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $V_{\mathrm{S}0}$ perturbation', loc='left')

distance = stats.wasserstein_distance(vals_st,vals_un)
median_distance = np.median(vals_st) - np.median(vals_un)
print("Wasserstein distance between the two distributions is %.3f" %distance)
print("Median distance between the two distributions is %.0f m/s" %median_distance)

plt.savefig(figpath+'ch6st_histograms.png',bbox_inches='tight',format='png')

In [ ]:
# seismic data QC

# pre-stimulation FWI
axes, ch6un_mute_group1 = sep.read_file(datapath+"ch6un_mute4_group1.H")
ch6un_mute_group1 = ch6un_mute_group1.reshape(axes.n,order='F').T

axes, ch6un_data_group1 = sep.read_file(datapath+"ch6un_data_group1.H.bp30150")
ch6un_data_group1 = ch6un_data_group1.reshape(axes.n,order='F').T * ch6un_mute_group1
ch6un_data_group1 = ch6un_data_group1[:,0:601,:]

axes, ch6un_model_syn = sep.read_file(datapath+"ch6un_model_3d.H.syn_hf.bp30150")
ch6un_model_syn = ch6un_model_syn.reshape(axes.n,order='F').T * ch6un_mute_group1
ch6un_model_syn = ch6un_model_syn[:,0:601,:]

axes, ch6un_fwi3d_10b = sep.read_file(datapath+"ch6un_fwi3d_10b.H.syn_hf.bp30150")
ch6un_fwi3d_10b = ch6un_fwi3d_10b.reshape(axes.n,order='F').T * ch6un_mute_group1
ch6un_fwi3d_10b = ch6un_fwi3d_10b[:,0:601,:]


# post-stimulation FWI
axes, ch6st_mute_group1 = sep.read_file(datapath+"ch6st_mute4_group1.H")
ch6st_mute_group1 = ch6st_mute_group1.reshape(axes.n,order='F').T

axes, ch6st_data_group1 = sep.read_file(datapath+"ch6un_data_group1.H.bp30150")
ch6st_data_group1 = ch6st_data_group1.reshape(axes.n,order='F').T * ch6st_mute_group1
ch6st_data_group1 = ch6st_data_group1[:,600:,:]

axes, ch6st_model_syn = sep.read_file(datapath+"ch6st_model_3d.H.syn_hf.bp30150")
ch6st_model_syn = ch6st_model_syn.reshape(axes.n,order='F').T * ch6st_mute_group1
ch6st_model_syn = ch6st_model_syn[:,600:,:]

axes, ch6st_fwi3d_10b = sep.read_file(datapath+"ch6st_fwi3d_10b.H.syn_hf.bp30150")
ch6st_fwi3d_10b = ch6st_fwi3d_10b.reshape(axes.n,order='F').T * ch6st_mute_group1
ch6st_fwi3d_10b = ch6st_fwi3d_10b[:,600:,:]

In [ ]:
plotseis_interleave(ch6un_data_group1[[9,6,3,0]], ch6un_model_syn[[9,6,3,0]], inter=50, valmin=-4, valmax=4, savefig="ch6un_model_3d_interleave")
plotseis_interleave(ch6un_data_group1[[9,6,3,0]], ch6un_fwi3d_10b[[9,6,3,0]], inter=50, valmin=-4, valmax=4, savefig="ch6un_fwi3d_10b_interleave")

In [ ]:
plotseis_interleave(ch6st_data_group1[[10,7,5,3]], ch6st_model_syn[[10,7,5,3]], inter=50, xmin=0, xmax=600, valmin=-4, valmax=4, savefig="ch6st_model_3d_interleave")
plotseis_interleave(ch6st_data_group1[[10,7,5,3]], ch6st_fwi3d_10b[[10,7,5,3]], inter=50, xmin=0, xmax=600, valmin=-4, valmax=4, savefig="ch6st_fwi3d_10b_interleave")